In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\TNF-α")

In [4]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [6]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [7]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [8]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [9]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7279 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [10]:
X_scaled_data

,MW,SpAbs_Dz(p),HyWi_Dz(p),H_Dz(p),AVS_Dz(p),Wi_Dz(p),SM6_Dz(m),SM5_Dz(m),SM4_Dz(m),SM3_Dz(m),...,nImidazoles,CATS2D_01_DA,nROCON,F10[C-Cl],B10[C-Cl],NaaS,H-048,SaaS,F06[N-S],Cl-089
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,0.100131,0.013547,0.507587,0.063662,0.013296,0.001367,0.527579,0.525839,0.525696,0.519653,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0625,0.419647,0.0,0.000000
138703696.0,0.125832,0.022601,0.560922,0.087584,0.022005,0.003041,0.589174,0.588881,0.586344,0.586353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.000000,0.0,0.000000
139316163.0,0.140863,0.032947,0.604614,0.081076,0.031646,0.004710,0.619311,0.618891,0.616900,0.616028,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0000,0.000000,0.0,0.000000
139315919.0,0.129659,0.025849,0.575313,0.077144,0.024839,0.003432,0.599286,0.599006,0.596526,0.596546,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0000,0.000000,0.0,0.000000
138703798.0,0.116939,0.019948,0.547034,0.076446,0.019465,0.002483,0.577807,0.577463,0.574934,0.574815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158020.0,0.109415,0.015771,0.521982,0.064281,0.015589,0.001713,0.546721,0.545390,0.544634,0.540213,...,0.0,0.0,0.0,0.3,1.0,0.0,0.0000,0.000000,0.0,0.166667
9631088.0,0.054518,0.006022,0.420121,0.030603,0.006026,0.000363,0.429805,0.427252,0.427849,0.418949,...,0.0,0.5,0.0,0.0,0.0,0.5,0.0625,0.432963,0.0,0.000000
3247362.0,0.109812,0.018124,0.540971,0.082993,0.018233,0.002262,0.578905,0.577664,0.576976,0.572792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.000000


In [11]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [12]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [13]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [14]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,MW,SpAbs_Dz(p),HyWi_Dz(p),H_Dz(p),AVS_Dz(p),Wi_Dz(p),SM6_Dz(m),SM5_Dz(m),SM4_Dz(m),SM3_Dz(m),...,nImidazoles,CATS2D_01_DA,nROCON,F10[C-Cl],B10[C-Cl],NaaS,H-048,SaaS,F06[N-S],Cl-089
cid,,,,,,,,,,,,,,,,,,,,,
5273,206.26,118.380642,7.140023,36.564879,54.146667,404.000006,21.071030,17.523405,14.091411,10.406821,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13463,285.72,190.494874,8.062983,48.415156,88.434501,836.924108,23.817407,19.815686,15.919192,11.789137,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39148,294.36,230.378561,8.412423,53.766255,106.971429,1119.600009,25.147185,20.926516,16.803419,12.460093,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
56328,253.27,187.873039,8.019371,41.812397,85.250873,763.486407,23.353702,19.430187,15.609420,11.558371,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
67358,283.28,223.797303,8.330601,47.075177,101.491334,1010.413342,24.297929,20.221731,16.234628,12.042389,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7279 Moleculars × 1261 Descriptors


,MW,SpAbs_Dz(p),HyWi_Dz(p),H_Dz(p),AVS_Dz(p),Wi_Dz(p),SM6_Dz(m),SM5_Dz(m),SM4_Dz(m),SM3_Dz(m),...,nImidazoles,CATS2D_01_DA,nROCON,F10[C-Cl],B10[C-Cl],NaaS,H-048,SaaS,F06[N-S],Cl-089
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.044327,0.004352,0.377627,0.021600,0.004180,0.000221,0.383511,0.381923,0.380461,0.375382,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0
13463,0.064546,0.007027,0.430884,0.027621,0.006829,0.000459,0.434015,0.432684,0.431088,0.427043,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0
39148,0.066744,0.008507,0.451048,0.030340,0.008261,0.000614,0.458469,0.457283,0.455580,0.452118,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0
56328,0.056289,0.006930,0.428368,0.024266,0.006583,0.000418,0.425488,0.424147,0.422508,0.418418,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0
67358,0.063925,0.008262,0.446326,0.026940,0.007838,0.000554,0.442852,0.441676,0.439826,0.436507,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.0
